<a href="https://colab.research.google.com/github/Ezra-A-Cohen/Webscraping/blob/main/Copy_of_Bozu_Deliverable_1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install requests
!pip install beautifulsoup4
!pip install lxml
!pip install requests

In [ ]:
import requests
from bs4 import BeautifulSoup as soup
from urllib.request import urlopen as uReq
import csv
import pandas
from lxml import html

In [ ]:
page = requests.get("https://en.wikipedia.org/wiki/Donald_Trump").text

I was strugggling with beautiful soup but I found the xpath method to be easier than beautifulsoup because it can find text with just one line of code. You just list all the tags and attributes to get to the element you're looking for. 

In [ ]:
tree = html.fromstring(page)

extracting the data for Donald Trump

In [ ]:
name = tree.xpath("//table[@class='infobox vcard']/tbody/tr/td[@class='infobox-data']/div[@class='nickname']/text()")[0]
print(name)

Donald John Trump


In [ ]:
politician = tree.xpath("//table[@class='infobox vcard']/tbody/tr/th[@class='infobox-above']/div[@class='fn']/text()") [0]
print(politician)

Donald Trump


In [ ]:
tempplace = tree.xpath("//table[@class='infobox vcard']/tbody/tr/th[text()='Born']/../td/a/text()")[0]
bdayloc = tree.xpath("//table[@class='infobox vcard']/tbody/tr/th[text()='Born']/../td[@class='infobox-data']/text()")
place = tempplace + bdayloc[1]
place = place.replace(",", "")
print(place)

Queens New York City U.S.


In [ ]:
bday = bdayloc[0]
bday = bday.replace(",", "")
print(bday)

June 14 1946


In [ ]:
party = tree.xpath("//table[@class='infobox vcard']/tbody/tr/th[text()='Political party']/../td/a/text()")[0]
print(party)

Republican


In [ ]:
professions = tree.xpath("//table[@class='infobox vcard']/tbody/tr/th[contains(text(),'Profession') or contains(text(),'Occupation')]/../td/div/ul/li/text()")
other_professions = ""
print(professions)
for occupation in professions:
  if not "politician" in occupation.lower():
    other_professions = other_professions + occupation.replace("'","") + " - "
print(other_professions)

['Politician', 'businessman', 'television presenter']
businessman - television presenter - 


In [ ]:

temp = tree.xpath("//table[@class='infobox vcard']/tbody/tr/th/a/text()")[0]
print(temp)
if "vice" in temp.lower():
  type = "Vice President" 
else: 
  type = "President"
print(type)

President of the United States
President


In [ ]:
dict = {
    "politician" : politician,
    "name" : name,
    "bday" : bday,
    "place" : place,
    "party" : party,
}
print(dict)

{'politician': 'Donald Trump', 'name': 'Donald John Trump', 'bday': 'June 14 1946', 'place': 'Queens New York City U.S.', 'party': 'Republican'}


function that takes in a politician's page and outputs the scraped info as a dictionary.

In [ ]:
def scrape(url):
  page = requests.get(url).text
  tree = html.fromstring(page)
  name = tree.xpath("//table[@class='infobox vcard']/tbody/tr/td[@class='infobox-data']/div[@class='nickname']/text()")
  politician = tree.xpath("//table[@class='infobox vcard']/tbody/tr/th[@class='infobox-above']/div[@class='fn']/text()")[0]
  tempplace = tree.xpath("//table[@class='infobox vcard']/tbody/tr/th[text()='Born']/../td/a/text()")
  bdayloc = tree.xpath("//table[@class='infobox vcard']/tbody/tr/th[text()='Born']/../td[@class='infobox-data']/text()")
  place = ""
  professions = tree.xpath("//table[@class='infobox vcard']/tbody/tr/th[contains(text(),'Profession') or contains(text(),'Occupation')]/../td/div/ul/li/text()")
  other_professions = ""
  for occupation in professions:
    if not "politician" in occupation.lower():
      other_professions = other_professions + occupation.replace("'","") + " - "
    else:
      professions = ""
  #place = tempplace + bdayloc[1]
  #place = place.replace(",", "")
  #bday = bdayloc[0]
  #bday = bday.replace(",", "")
  party = tree.xpath("//table[@class='infobox vcard']/tbody/tr/th[text()='Political party']/../td/a/text()")
  temp = tree.xpath("//table[@class='infobox vcard']/tbody/tr/th/a/text()")[0]
  if "vice" in temp.lower():
    type = "Vice President" 
  else: 
    type = "President"
  if len(party)>0:
    party = party[0]
  else:
    party = ""
  if len(name)>0:
    name = name[0]
  else:
    t = url.split("/")
    name=t[len(t)-1].replace("_"," ")
  if len(tempplace)>0:
    place = tempplace[0] + bdayloc[1]
    place = place.replace(",", "")
  #else:
    #place = bdayloc[0]
  bday = bdayloc[0]
  bday = bday.replace(",", "")
  dict = {
    "politician" : politician,
    "name" : name,
    "bday" : bday,
    "place" : place,
    "party" : party,
    "professions" : other_professions,
    "type" : type 
  }
  return dict

testing the scrape function

In [ ]:
print(scrape("https://en.wikipedia.org/wiki/George_H._W._Bush"))

{'politician': 'George H. W. Bush', 'name': 'George Herbert Walker Bush', 'bday': 'June 12 1924', 'place': 'Milton Massachusetts U.S.', 'party': 'Republican', 'professions': 'diplomat - businessman - ', 'type': 'President'}


getting all the links to each politician's page from the articles and putting them in an array

In [ ]:
politicians=["https://en.wikipedia.org/wiki/Category:21st-century_presidents_of_the_United_States",
"https://en.wikipedia.org/wiki/Category:20th-century_presidents_of_the_United_States",
"https://en.wikipedia.org/wiki/Category:19th-century_presidents_of_the_United_States",
"https://en.wikipedia.org/wiki/Category:21st-century_vice_presidents_of_the_United_States",
"https://en.wikipedia.org/wiki/Category:20th-century_vice_presidents_of_the_United_States",
"https://en.wikipedia.org/wiki/Category:19th-century_vice_presidents_of_the_United_States"
]

def getUrls(url):
  urls=[]
  page = requests.get(url).text
  tree = html.fromstring(page)
  turls = tree.xpath("//div[@id='bodyContent']/div[@id='mw-content-text']/div[@class='mw-category-generated']/div/div/ul/li/a/@href")
  if(len(turls)>0):
    for turl in turls:
      if "sandbox" not in turl:
        urls.append("https://en.wikipedia.org/"+turl)
  else:
    turls = tree.xpath("//div[@id='bodyContent']/div[@id='mw-content-text']/div[@class='mw-category-generated']/div/div/div/div/ul/li/a/@href")
    for turl in turls:
      if "sandbox" not in turl:
        urls.append("https://en.wikipedia.org/"+turl)
  return urls

urls = []
for politician in politicians:
  tempurls = urls
  urls = tempurls + getUrls(politician)


looping the scrape function through the array of all the politicians to scrape each politician's webpage

In [ ]:
counter=1
mydict={}
for x in urls:
  mydict[counter]=scrape(x)
  counter=counter+1
print(mydict)


{1: {'politician': 'Joe Biden', 'name': 'Joseph Robinette Biden Jr.', 'bday': 'November 20 1942', 'place': 'Scranton Pennsylvania U.S.', 'party': 'Democratic', 'professions': 'lawyer - author - ', 'type': 'President'}, 2: {'politician': 'George W. Bush', 'name': 'George Walker Bush', 'bday': 'July 6 1946', 'place': 'New Haven Connecticut U.S.', 'party': 'Republican', 'professions': 'businessman - ', 'type': 'President'}, 3: {'politician': 'Barack Obama', 'name': 'Barack Hussein Obama II', 'bday': 'August 4 1961', 'place': 'Honolulu Hawaii U.S.', 'party': 'Democratic', 'professions': 'lawyer - author - ', 'type': 'President'}, 4: {'politician': 'Donald Trump', 'name': 'Donald John Trump', 'bday': 'June 14 1946', 'place': 'Queens New York City U.S.', 'party': 'Republican', 'professions': 'businessman - television presenter - ', 'type': 'President'}, 5: {'politician': 'George H. W. Bush', 'name': 'George Herbert Walker Bush', 'bday': 'June 12 1924', 'place': 'Milton Massachusetts U.S.', '

turning the scraped data into a dataframe and making a csv

In [ ]:
df = pandas.DataFrame(mydict)
df_t = df.T
#to transpose rows and columns
csvfile = df_t.to_csv('df.csv', index=False)

pandas.read_csv('df.csv')


,politician,name,bday,place,party,professions,type
0,Joe Biden,Joseph Robinette Biden Jr.,November 20 1942,Scranton Pennsylvania U.S.,Democratic,lawyer - author -,President
1,George W. Bush,George Walker Bush,July 6 1946,New Haven Connecticut U.S.,Republican,businessman -,President
2,Barack Obama,Barack Hussein Obama II,August 4 1961,Honolulu Hawaii U.S.,Democratic,lawyer - author -,President
3,Donald Trump,Donald John Trump,June 14 1946,Queens New York City U.S.,Republican,businessman - television presenter -,President
4,George H. W. Bush,George Herbert Walker Bush,June 12 1924,Milton Massachusetts U.S.,Republican,diplomat - businessman -,President
...,...,...,...,...,...,...,...
88,Daniel D. Tompkins,Daniel D. Tompkins,June 21 1774,Scarsdale,Democratic-Republican,NaN,Vice President
89,John Tyler,John Tyler,March 29 1790,Greenway Plantation,Independent,lawyer -,President
90,Martin Van Buren,Maarten van Buren,December 5 1782,Kinderhook New York U.S.,NaN,lawyer -,President
91,William A. Wheeler,William Almon Wheeler,June 30 1819,Malone New York U.S.,Republican,NaN,Vice President


In [ ]:
turls = ['https://en.wikipedia.org//wiki/Joe_Biden', 'https://en.wikipedia.org//wiki/George_W._Bush', 'https://en.wikipedia.org//wiki/Barack_Obama', 'https://en.wikipedia.org//wiki/Donald_Trump', 'https://en.wikipedia.org//wiki/George_H._W._Bush', 'https://en.wikipedia.org//wiki/Jimmy_Carter', 'https://en.wikipedia.org//wiki/Bill_Clinton', 'https://en.wikipedia.org//wiki/Calvin_Coolidge', 'https://en.wikipedia.org//wiki/Dwight_D._Eisenhower', 'https://en.wikipedia.org//wiki/Gerald_Ford', 'https://en.wikipedia.org//wiki/Warren_G._Harding', 'https://en.wikipedia.org//wiki/Herbert_Hoover', 'https://en.wikipedia.org//wiki/Lyndon_B._Johnson', 'https://en.wikipedia.org//wiki/John_F._Kennedy', 'https://en.wikipedia.org//wiki/William_McKinley', 'https://en.wikipedia.org//wiki/Richard_Nixon', 'https://en.wikipedia.org//wiki/Ronald_Reagan', 'https://en.wikipedia.org//wiki/Franklin_D._Roosevelt', 'https://en.wikipedia.org//wiki/Theodore_Roosevelt', 'https://en.wikipedia.org//wiki/William_Howard_Taft', 'https://en.wikipedia.org//wiki/Harry_S._Truman', 'https://en.wikipedia.org//wiki/Woodrow_Wilson', 'https://en.wikipedia.org//wiki/John_Adams', 'https://en.wikipedia.org//wiki/John_Quincy_Adams', 'https://en.wikipedia.org//wiki/Chester_A._Arthur', 'https://en.wikipedia.org//wiki/James_Buchanan', 'https://en.wikipedia.org//wiki/Grover_Cleveland', 'https://en.wikipedia.org//wiki/Millard_Fillmore', 'https://en.wikipedia.org//wiki/James_A._Garfield', 'https://en.wikipedia.org//wiki/Ulysses_S._Grant', 'https://en.wikipedia.org//wiki/Benjamin_Harrison', 'https://en.wikipedia.org//wiki/William_Henry_Harrison', 'https://en.wikipedia.org//wiki/Rutherford_B._Hayes', 'https://en.wikipedia.org//wiki/Andrew_Jackson', 'https://en.wikipedia.org//wiki/Thomas_Jefferson', 'https://en.wikipedia.org//wiki/Andrew_Johnson', 'https://en.wikipedia.org//wiki/Abraham_Lincoln', 'https://en.wikipedia.org//wiki/James_Madison', 'https://en.wikipedia.org//wiki/William_McKinley', 'https://en.wikipedia.org//wiki/James_Monroe', 'https://en.wikipedia.org//wiki/Franklin_Pierce', 'https://en.wikipedia.org//wiki/James_K._Polk', 'https://en.wikipedia.org//wiki/Zachary_Taylor', 'https://en.wikipedia.org//wiki/John_Tyler', 'https://en.wikipedia.org//wiki/Martin_Van_Buren', 'https://en.wikipedia.org//wiki/Joe_Biden', 'https://en.wikipedia.org//wiki/Dick_Cheney', 'https://en.wikipedia.org//wiki/Al_Gore', 'https://en.wikipedia.org//wiki/Kamala_Harris', 'https://en.wikipedia.org//wiki/Mike_Pence', 'https://en.wikipedia.org//wiki/Spiro_Agnew', 'https://en.wikipedia.org//wiki/Alben_W._Barkley', 'https://en.wikipedia.org//wiki/George_H._W._Bush', 'https://en.wikipedia.org//wiki/Calvin_Coolidge', 'https://en.wikipedia.org//wiki/Charles_Curtis', 'https://en.wikipedia.org//wiki/Charles_G._Dawes', 'https://en.wikipedia.org//wiki/Charles_W._Fairbanks', 'https://en.wikipedia.org//wiki/Gerald_Ford', 'https://en.wikipedia.org//wiki/John_Nance_Garner', 'https://en.wikipedia.org//wiki/Al_Gore', 'https://en.wikipedia.org//wiki/Hubert_Humphrey', 'https://en.wikipedia.org//wiki/Lyndon_B._Johnson', 'https://en.wikipedia.org//wiki/Thomas_R._Marshall', 'https://en.wikipedia.org//wiki/Walter_Mondale', 'https://en.wikipedia.org//wiki/Richard_Nixon', 'https://en.wikipedia.org//wiki/Dan_Quayle', 'https://en.wikipedia.org//wiki/Nelson_Rockefeller', 'https://en.wikipedia.org//wiki/Theodore_Roosevelt', 'https://en.wikipedia.org//wiki/James_S._Sherman', 'https://en.wikipedia.org//wiki/Harry_S._Truman', 'https://en.wikipedia.org//wiki/Henry_A._Wallace', 'https://en.wikipedia.org//wiki/John_C._Breckinridge', 'https://en.wikipedia.org//wiki/Aaron_Burr', 'https://en.wikipedia.org//wiki/John_C._Calhoun', 'https://en.wikipedia.org//wiki/George_Clinton_(vice_president)', 'https://en.wikipedia.org//wiki/Schuyler_Colfax', 'https://en.wikipedia.org//wiki/George_M._Dallas', 'https://en.wikipedia.org//wiki/Millard_Fillmore', 'https://en.wikipedia.org//wiki/Elbridge_Gerry', 'https://en.wikipedia.org//wiki/Hannibal_Hamlin', 'https://en.wikipedia.org//wiki/Thomas_A._Hendricks', 'https://en.wikipedia.org//wiki/Garret_Hobart', 'https://en.wikipedia.org//wiki/Thomas_Jefferson', 'https://en.wikipedia.org//wiki/Andrew_Johnson', 'https://en.wikipedia.org//wiki/Richard_Mentor_Johnson', 'https://en.wikipedia.org//wiki/William_R._King', 'https://en.wikipedia.org//wiki/Levi_P._Morton', 'https://en.wikipedia.org//wiki/Adlai_Stevenson_I', 'https://en.wikipedia.org//wiki/Daniel_D._Tompkins', 'https://en.wikipedia.org//wiki/John_Tyler', 'https://en.wikipedia.org//wiki/Martin_Van_Buren', 'https://en.wikipedia.org//wiki/William_A._Wheeler', 'https://en.wikipedia.org//wiki/Henry_Wilson']
for surl in turls: 
  page = requests.get(surl).text
  tree = html.fromstring(page)
 # professions = tree.xpath("//table[@class='infobox vcard']/tbody/tr/th[text()='Occupation']/../td/div/ul/li/text()") 
  professions = tree.xpath("//table[@class='infobox vcard']/tbody/tr/th[contains(text(),'Profession') or contains(text(),'Occupation')]/../td/div/ul/li/text()")
  other_professions = ""
  for occupation in professions:
    if not "politician" in occupation.lower():
      other_professions = other_professions + occupation.replace("'","") + " - "
  print(surl +"         " + other_professions)

https://en.wikipedia.org//wiki/Joe_Biden         lawyer - author - 
https://en.wikipedia.org//wiki/George_W._Bush         businessman - 
https://en.wikipedia.org//wiki/Barack_Obama         lawyer - author - 
https://en.wikipedia.org//wiki/Donald_Trump         businessman - television presenter - 
https://en.wikipedia.org//wiki/George_H._W._Bush         diplomat - businessman - 
https://en.wikipedia.org//wiki/Jimmy_Carter         Philanthropist - author - 
https://en.wikipedia.org//wiki/Bill_Clinton         lawyer - professor - author - 
https://en.wikipedia.org//wiki/Calvin_Coolidge         lawyer - 
https://en.wikipedia.org//wiki/Dwight_D._Eisenhower         Military officer - 
https://en.wikipedia.org//wiki/Gerald_Ford         lawyer - 
https://en.wikipedia.org//wiki/Warren_G._Harding         journalist - 
https://en.wikipedia.org//wiki/Herbert_Hoover         engineer - 
https://en.wikipedia.org//wiki/Lyndon_B._Johnson         teacher - 
https://en.wikipedia.org//wiki/John_F._Kennedy